# Creating a generative AI with `twinLab`

In the language of AI models, the word `generative` means that we train a model to learn a statistical distribution of $y$, rather than just a point estimate of $y$. In this way, the Gaussian Processes used by `twinLab` are naturally generative models, and we can use these to make a crude generative AI model for images.

In [ ]:
# Standard imports
import pickle

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# twinLab
import twinlab as tl

In [ ]:
# Parameters

# Experiment
# experiment = "MNIST-lowres" # 8x8 images
experiment = "MNIST" # 28x28 images
# experiment = "CIFAR-10" # https://www.cs.toronto.edu/~kriz/cifar.html 

# Random numbers
random_seed = 123

# Training parameters
training_samples = 1500
explained_variance = 0.5
onehot_encode = True

In [ ]:
# Functions
def unpickle(file): 
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict

def wrangle_image(linear_image, npix): 
    # Reshape a CIFAR-10 image
    pix = npix**2
    if len(linear_image) == pix:
        image = linear_image.reshape(npix, npix)
    elif len(linear_image) == 3*pix:
        R = linear_image[0*pix:1*pix].reshape(npix, npix)
        G = linear_image[1*pix:2*pix].reshape(npix, npix)
        B = linear_image[2*pix:3*pix].reshape(npix, npix)
        image = np.dstack((R, G, B)).astype(np.uint8)
    else:
        raise ValueError("Image is neither 1D nor 3D.")
    return image

def plot_images(df, nrow, ncol, figx, figy, npix, vmin=0., vmax=255.):
    iplot = 0
    plt.subplots(nrow, ncol, figsize=(figx*ncol, figy*nrow))
    for _ in range(nrow):
        for _ in range(ncol):
            image = wrangle_image(df.iloc[iplot].to_numpy(), npix)
            iplot += 1
            plt.subplot(nrow, ncol, iplot)
            plt.imshow(image, cmap="binary_r", vmin=vmin, vmax=vmax)
            plt.xticks([]); plt.yticks([])
    plt.show()

In [ ]:
# Calculations
np.random.seed(random_seed)

In [ ]:
if experiment == "MNIST-lowres":

    # Read data in and set pixels that the training data has 
    # In this case, it's 8x8 pixel pictures of numbers 0 to 9. 1798 pictures
    npix = 8
    filepath = "MNIST-lowres/data.csv"
    df = pd.read_csv(filepath)

elif experiment == "MNIST":

    # 10,000 examples of 28x28 pixel pictures of numbers 0 to 9
    npix = 28
    filepath = "MNIST/test.csv"
    df = pd.read_csv(filepath)
    df.rename({"label": "number"}, axis="columns", inplace=True)

elif experiment == "CIFAR-10":

    # 32x32 pixel pictures. 10 pictures of 10 different types of object 
    npix = 32
    filepath = "CIFAR-10/data_batch_1"
    data = unpickle(filepath)

    # Iterate through the RGB values that compose these pictures 
    # Each pixel gets a value so we can unpack a 3D object into the 2D dataframe
    df = pd.DataFrame(data[b"data"])
    df.columns = [f"{RGB}-{i}-{j}" for RGB in ["R", "G", "B"] for i in range(npix) for j in range(npix)]
    df["number"] = data[b"labels"] # TODO: Try to insert this as the first column
    image_dict = {
        "airplane": 0,
        "automobile": 1,
        "bird": 2,
        "cat": 3,
        "deer": 4,
        "dog": 5,
        "frog": 6,
        "horse": 7,
        "ship": 8,
        "truck": 9
    }
    df = df[df["number"] == image_dict["dog"]] # TODO: Only dogs for tests!

else:
    raise ValueError("Experiement not recognised")

# One-hot encoding
if onehot_encode:
    df = pd.get_dummies(df, columns=["number"])

# Set up campaign
if onehot_encode:
    inputs = [f"number_{i}" for i in range(10)]
else:
    inputs = ["number"]
outputs = list(df.drop(columns=inputs))

# Print to screen
display(df)

# Plot an image of the data
nrow, ncol = 10, 10
figx, figy = 1.2, 1.2
plot_images(df[outputs], nrow, ncol, figx, figy, npix)

In [ ]:
# Setup campaign
setup_dict = {
    "inputs": inputs,
    "outputs": outputs,
    'estimator': 'gaussian_process_regression', # What type of model do you want to use? 
    'decompose_outputs': True, # Equivalent of PCA/SVD; necessary to learn structure
    'output_explained_variance': explained_variance # Toggle this number to improve accuracy
}
campaign = tl.Campaign(**setup_dict)

# Train campaign
train_dict = {
    "df": df,
    "train_test_split": training_samples, # How many rows are used for training?
}
campaign.fit(**train_dict)

In [ ]:
# Show some diagnostic information about the trained campaign
campaign.get_diagnostics()

In [ ]:
# Make some predictions
df_predict = pd.DataFrame({'number': list(range(10))})
if onehot_encode:
    df_predict = pd.get_dummies(df_predict, columns=["number"])
display(df_predict)

# Pull out the mean prediction
df_mean, _ = campaign.predict(df_predict)
display(df_mean)

# Plot the mean value of each figure/number from the trained dataset
nrow, ncol = 2, 5
figx, figy = 2, 2
plot_images(df_mean, nrow, ncol, figx, figy, npix)

In [ ]:
# Generate some weird numbers
if onehot_encode:
    nrow = 90
    df_predict = pd.DataFrame(np.zeros((nrow, 10)))
    df_predict.columns = [f"number_{icol}" for icol in range(10)]
    for irow in range(nrow):
        icol = irow % 10
        inum1 = irow // 10
        inum2 = inum1 + 1
        ifrac = icol/10
        ifrac1, ifrac2 = 1-ifrac, ifrac
        df_predict[f"number_{inum1}"].iloc[irow] = ifrac1
        df_predict[f"number_{inum2}"].iloc[irow] = ifrac2
else:
    df_predict = pd.DataFrame({'number': np.linspace(0, 8.9, 90)})
display(df_predict)

# Make predictions
df_mean, _ = campaign.predict(df_predict)
display(df_mean)

# Plot
nrow, ncol = 9, 10
figx, figy = 1.2, 1.2
plot_images(df_mean, nrow, ncol, figx, figy, npix)

In [ ]:
# Generate samples from the trained model
nsample = 10
df_predict = pd.DataFrame({'number': list(range(10))})
if onehot_encode:
    df_predict = pd.get_dummies(df_predict, columns=["number"])
display(df_predict)

# Generate samples
df_samples = campaign.sample(df_predict, nsample, {"observation_noise": True}) # NOTE: "observation_noise" needs to be true in order to generate diverse samples
df_samples = df_samples.stack(level=1).reset_index(1).rename(columns={"level_1": "sample"})
display(df_samples)

# Plot
nrow, ncol = 10, nsample
figx, figy = 1.2, 1.2
plot_images(df_samples[outputs], nrow, ncol, figx, figy, npix)